<a href="https://colab.research.google.com/github/TheAgaveFairy/MusicGeneration/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/arnavmishra6996/music-generation-using-decoder-only-model-gpt2

this is the inspiration, more or less copying it but in my own typing + modifications so i understand it

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
! pip install pretty_midi
import pretty_midi

In [3]:
! pip install transformers

In [4]:
"""
!pip install -q kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

! kaggle competitions download -c 'name-of-competition'
! mkdir train
! unzip train.zip -d train
"""

"\n!pip install -q kaggle\nfrom google.colab import files\nfiles.upload()\n\n! mkdir ~/.kaggle\n\n! cp kaggle.json ~/.kaggle/\n! chmod 600 ~/.kaggle/kaggle.json\n#! kaggle datasets list\n\n! kaggle competitions download -c 'name-of-competition'\n! mkdir train\n! unzip train.zip -d train\n"

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imsparsh/musicnet-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1


In [5]:
"""for dirname, _, filenames in os.walk(path):
    for filename in filenames[:10]: # just make sure this is working
        print(os.path.join(dirname, filename))
"""
def list_files(startpath, trunc=True):
    print(f"root path is:\n{path}\nbeginning walk:")
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        if trunc:
            files = files[:2]
        for f in files:
            print('{}{}'.format(subindent, f))
list_files(path)

root path is:
/root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1
beginning walk:
1/
    musicnet_metadata.csv
    musicnet.npz
    musicnet_midis/
        musicnet_midis/
            Ravel/
                2178_gr_rqtf2.mid
                2179_gr_rqtf3.mid
            Cambini/
                2080_quint2d3.mid
                2078_quint2d1.mid
            Beethoven/
                2336_vns08_3.mid
                2376_qt08_1.mid
            Schubert/
                1730_schubert_op114_5.mid
                1758_d958-2.mid
            Mozart/
                1791_kv_465_4.mid
                1859_kv_464_2.mid
            Faure/
                2166_gr_f45m1.mid
                2167_gr_f45m2.mid
            Bach/
                2239_fugue17.mid
                2284_vhs1_3.mid
            Haydn/
                2104_op64n5_1.mid
                2105_op64n5_2.mid
            Brahms/
                2159_bra40_2.mid
                2151_br25m4.mid
            Dvorak/
 

In [7]:
temp_file_path = os.path.join(path, 'musicnet/musicnet/test_labels/2382.csv')
#print(temp_file_path)
data = pd.read_csv(temp_file_path)
data.head(3)

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,10206,42462,41,61,0.0,1.489583,Dotted Half
1,10206,25054,41,65,0.0,0.489583,Quarter
2,10206,22494,43,46,0.0,0.333333,Dotted Eighth


In [8]:
# let's just pick a file and see what we're dealing with
midi_data = pretty_midi.PrettyMIDI(os.path.join(path,'musicnet_midis/musicnet_midis/Beethoven/2313_qt15_1.mid'))

/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [9]:
"""
data = []
for instrument in midi_data.instruments:
    #print(instrument) # Instrument(program=40, is_drum=False, name="Violin1") kinda output
    for note in instrument.notes:
        #print(note) # start, end, pitch, velocity
        velocity = note.velocity
        start_time = note.start
        end_time = note.end
        instrument_name = instrument.name if instrument.name else 'Unnamed'
        note_name = pretty_midi.note_number_to_name(note.pitch)
        start_beat = midi_data.get_beats(start_time)
        end_beat = midi_data.get_beats(end_time)
        note_value = note.pitch

        data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, velocity, note_value])

# Create a DataFrame
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'velocity', 'note_value']
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file (optional)
#df.to_csv('midi_data.csv', index=False)

print(df.head())
"""

'\ndata = []\nfor instrument in midi_data.instruments:\n    #print(instrument) # Instrument(program=40, is_drum=False, name="Violin1") kinda output\n    for note in instrument.notes:\n        #print(note) # start, end, pitch, velocity\n        velocity = note.velocity\n        start_time = note.start\n        end_time = note.end\n        instrument_name = instrument.name if instrument.name else \'Unnamed\'\n        note_name = pretty_midi.note_number_to_name(note.pitch)\n        start_beat = midi_data.get_beats(start_time)\n        end_beat = midi_data.get_beats(end_time)\n        note_value = note.pitch\n\n        data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, velocity, note_value])\n\n# Create a DataFrame\ncolumns = [\'start_time\', \'end_time\', \'instrument\', \'note\', \'start_beat\', \'end_beat\', \'velocity\', \'note_value\']\ndf = pd.DataFrame(data, columns=columns)\n\n# Save the DataFrame to a CSV file (optional)\n#df.to_csv(\'midi_data.cs

In [6]:
entire_data = []
from tqdm import tqdm
def midi_data_extraction(file_path):
    # Check if the path is a directory
    if os.path.isdir(file_path):
        # Loop through all files in the directory
        for file_name in tqdm(os.listdir(file_path)):
            if file_name.endswith('.mid'):
                # Construct the full file path
                full_path = os.path.join(file_path, file_name)
                # Load the MIDI file
                midi_data = pretty_midi.PrettyMIDI(full_path)

                # Extract data for each note
                for instrument in midi_data.instruments:
                    for note in instrument.notes:
                        start_time = note.start
                        end_time = note.end
                        instrument_name = instrument.name if instrument.name else 'Unnamed'
                        note_name = pretty_midi.note_number_to_name(note.pitch)
                        start_beat = midi_data.get_beats(start_time)
                        end_beat = midi_data.get_beats(end_time)
                        note_value = note.pitch
                        duration = end_time - start_time  # Duration of the note
                        velocity = note.velocity  # Velocity of the note

                        # Add the data to the list
                        entire_data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, note_value, duration, velocity])

In [13]:
# path = '/root/.cache/kagglehub/datasets/imsparsh/musicnet-dataset/versions/1'
file_name = os.path.join(path, 'musicnet_midis/musicnet_midis/Beethoven')
# print(file_name)
midi_data_extraction(file_name)
entire_data[:1]

  0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
100%|██████████| 157/157 [46:39<00:00, 17.83s/it]


[[3.749994,
  3.9019034791666667,
  'Violin',
  'D5',
  array([  3.749994 ,   4.16666  ,   4.583326 ,   4.999992 ,   5.416658 ,
           5.833324 ,   6.24999  ,   6.666656 ,   7.083322 ,   7.499988 ,
           7.916654 ,   8.33332  ,   8.749986 ,   9.166652 ,   9.595223 ,
          10.04658  ,  10.508118 ,  10.924784 ,  11.34145  ,  11.758116 ,
          12.174782 ,  12.591448 ,  13.008114 ,  13.42478  ,  13.841446 ,
          14.258112 ,  14.674778 ,  15.091444 ,  15.50811  ,  15.924776 ,
          16.353347 ,  16.804704 ,  17.266242 ,  17.682908 ,  18.099574 ,
          18.51624  ,  18.932906 ,  19.349572 ,  19.766238 ,  20.182904 ,
          20.59957  ,  21.016236 ,  21.432902 ,  21.849568 ,  22.266234 ,
          22.6829   ,  23.099566 ,  23.537732 ,  23.985493 ,  24.402159 ,
          24.818825 ,  25.235491 ,  25.652157 ,  26.068823 ,  26.485489 ,
          26.902155 ,  27.318821 ,  27.735487 ,  28.152153 ,  28.568819 ,
          28.985485 ,  29.402151 ,  29.818817 ,  30.256983

In [7]:
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'note_value', 'duration', 'velocity']

# Create a DataFrame with the collected data
df = pd.DataFrame(entire_data, columns=columns)

# Save the DataFrame to a CSV file (optional)
#df.to_csv('entire_midi_data.csv', index=False)

# Print the first few rows of the DataFrame
print(df.head())

Empty DataFrame
Columns: [start_time, end_time, instrument, note, start_beat, end_beat, note_value, duration, velocity]
Index: []


GPT-2 needs this as text

In [15]:
with open('midi_text_data.txt', 'w') as file:
    file.write('<|startoftext|>\n')
    for index, row in tqdm(df.iterrows()):
        text_line = f"start_time: {row['start_time']}, end_time: {row['end_time']}, instrument: {row['instrument']}, note: {row['note']}, start_beat: {row['start_beat']}, end_beat: {row['end_beat']}, note_value: {row['note_value']}, duration: {row['duration']}, velocity: {row['velocity']}\n"
        file.write(text_line)
    file.write('<|endoftext|>\n')

Here we go

In [8]:
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            input_ids = ids[i:i+block_size]
            labels = ids[i+1:i+block_size+1] # shifted by 1
            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100

            yield {
                "input_ids": input_ids,#ids[i:i+block_size],
                "attention_mask": [1] * block_size,
                "labels": labels
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(block_size,), dtype=tf.int32)
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset('midi_text_data.txt', tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
#!pip install transformers[tf-cpu]

In [18]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir "/content/drive/My Drive/MusicGen"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/MusicGen’: File exists


In [ ]:
"""
with open('/content/drive/My Drive/MusicGen/midi_text_data.txt', 'w') as f:
  with open('midi_text_data.txt', 'r') as f2:
    f.write(f2.read())
  #f.write('./midi_text_data.txt')
"""

In [11]:
tf.config.list_physical_devices('CPU'),tf.config.list_physical_devices('GPU')

([PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')], [])

In [ ]:
"""
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, TFTrainingArguments #TFTrainer,
from transformers import DataCollatorForLanguageModeling
from transformers import TFTrainer# as TFTrainer
from tqdm import tqdm

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Define the training arguments
training_args = TFTrainingArguments(
    output_dir="./gpt2-midi",  # Directory where the model and tokenizer will be saved
    overwrite_output_dir=True,
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size for training
    save_steps=10_000,  # Save every 10,000 steps
    save_total_limit=2,  # Keep only the 2 most recent models
    prediction_loss_only=True,  # Only compute the loss during training
    logging_dir='./logs',  # Directory for logs
    logging_steps=500  # Log every 500 steps
)

#Gemini
strategy = tf.distribute.MirroredStrategy(['CPU:0'])

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    #/Gemini

# Initialize the Trainer
trainer = TFTrainer(
    model=model,#TFGPT2LMHeadModel.from_pretrained('gpt2'),
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)
"""
'This code was deprecated on or about 2021.'

In [ ]:
"""
# Start training
trainer.train()

# Save the model and tokenizer
trainer.save_model('./gpt2-midi')
tokenizer.save_pretrained('./gpt2-midi')
"""

In [16]:
#Claude 3.5Sonnet to the rescue, maybe
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling, DefaultDataCollator
from tqdm import tqdm
import os

# Initialize tokenizer and data collator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
"""
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True # changed from False
)
"""
data_collator = DefaultDataCollator()

# Training configuration
class TrainingConfig:
    output_dir = "./gpt2-midi"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    save_steps = 10_000
    save_total_limit = 2
    logging_dir = './logs'
    logging_steps = 500

config = TrainingConfig()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

# Set up distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')

    # Set up the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss
    )

# Prepare the dataset
def prepare_dataset_for_training(dataset, batch_size):
    """Convert dataset to tf.data.Dataset format"""
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    """
    # Modify attention mask shape
    dataset = dataset.map(lambda x: {
        **x,
        "attention_mask": tf.reshape(x["attention_mask"], (tf.shape(x["attention_mask"])[0], 1, 1, tf.shape(x["attention_mask"])[2]))  # Reshape to (batch_size, 1, 1, seq_len)
    })
    """
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset_for_training(
    train_dataset,  # Your original train_dataset
    config.per_device_train_batch_size
)

# Custom callback for model saving
class ModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_steps, output_dir, save_total_limit):
        super().__init__()
        self.save_steps = save_steps
        self.output_dir = output_dir
        self.save_total_limit = save_total_limit
        self.step = 0

    def on_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step % self.save_steps == 0:
            # Save the model
            checkpoint_dir = os.path.join(
                self.output_dir,
                f"checkpoint-{self.step}"
            )
            self.model.save_pretrained(checkpoint_dir)

            # Remove old checkpoints if exceeding limit
            checkpoints = sorted([
                d for d in os.listdir(self.output_dir)
                if d.startswith("checkpoint-")
            ])
            while len(checkpoints) > self.save_total_limit:
                checkpoint_to_remove = os.path.join(
                    self.output_dir,
                    checkpoints.pop(0)
                )
                tf.io.gfile.rmtree(checkpoint_to_remove)

# Set up callbacks
callbacks = [
    ModelCheckpoint(
        save_steps=config.save_steps,
        output_dir=config.output_dir,
        save_total_limit=config.save_total_limit
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=config.logging_dir,
        update_freq=config.logging_steps
    )
]

# Training
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=config.num_train_epochs,
    callbacks=callbacks
)

# Save the final model and tokenizer
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)
print(f"Model saved to {config.output_dir}")

Number of devices: 1


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Starting training...
Epoch 1/3


ValueError: in user code:

    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_tf_utils.py", line 1672, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.11/dist-packages/tf_keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filebt088adg.py", line 40, in tf__run_call_with_unpacked_inputs
        raise
    File "/tmp/__autograph_generated